# Temperature dependent elastic constants

## Background

$$C_{ijkl} = \frac{1}{V} \frac{\partial^2 U}{\partial \varepsilon_{ij}\partial \varepsilon_{kl}}$$

$$U(T) = \frac{V}{2}C_{ijkl}(T)\varepsilon_{ij}\varepsilon_{kl}$$

$$\sigma_{ij} = C_{ijkl}{\varepsilon_{kl}}$$

### How to get $U$ or $\sigma$

- MD
- Quasi-Harmonic

## Tasks

- Get $a_0$ from potential
- Lattice parameter (as a function of T)
  - MD
    - NVT
    - NPT
  - QH
- Calculate $U$ or $\sigma$ for various $\varepsilon$
  - MD: Equilibriate and average with LAMMPS
  - QH: Get strains from Yuriy's tool and run phonopy
- Fit

## Teams

- MD: Erik, Han, (Raynol), Prabhath, Jan
- QH: Raynol, (Sam), Bharathi, Ahmed, Haitham
- Fit & Yuriy: Sam
- Literature

# Implementation

* https://atomistics.readthedocs.io/en/latest/bulk_modulus_with_gpaw.html#elastic-matrix
* https://github.com/pyiron/atomistics/blob/main/tests/test_elastic_lammpslib_functional.py
* https://github.com/pyiron/pyiron_workflow_atomistics/blob/interstitials/pyiron_workflow_atomistics/dataclass_storage.py
* https://github.com/ligerzero-ai/pyiron_workflow_lammps/blob/main/pyiron_workflow_lammps/engine.py#L21

In [1]:
from ase.build import bulk
from ase.atoms import Atoms

In [ ]:
from atomistics.calculators import evaluate_with_lammpslib, get_potential_by_name

## Create bulk sample

In [3]:
structure = bulk('Al', 'fcc', a=4.05, cubic=True)

In [4]:
potential_name = "1999--Mishin-Y--Al--LAMMPS--ipr1"

In [5]:
df_pot_selected = get_potential_by_name(
    potential_name=potential_name
)


/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst


## 0K Relaxed Structure

In [6]:
def get_relaxed_structure(structure: Atoms, potential: str) -> Atoms:
    
    df_pot_selected = get_potential_by_name(
            potential_name=potential
        )
    
    result_dict = evaluate_with_lammpslib(
            task_dict={"optimize_positions_and_volume": structure},
            potential_dataframe=df_pot_selected,
        )
    
    structure_relaxed = result_dict['structure_with_optimized_positions_and_volume']

    return structure_relaxed

In [7]:
relaxed_structure = get_relaxed_structure(structure, potential_name)
relaxed_structure

/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
--------------------------------------------------------------------------

  Local host:   cmti001
  Local device: hfi1_0
--------------------------------------------------------------------------


Atoms(symbols='Al4', pbc=True, cell=[4.050004662201837, 4.050004662201837, 4.050004662201837])

## 0K Lattice Constant

In [8]:
def get_minimum_lattice_constant(structure: Atoms, potential: str) -> float:

    structure_relaxed = get_relaxed_structure(structure, potential)
    a_0 = structure_relaxed.get_volume()**(1/3) #Angstrom

    return a_0

In [9]:
a_0 = get_minimum_lattice_constant(structure, potential_name)
a_0

/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst


4.050004662201837

# Skeleton for the workflow

In [ ]:
def get_minimum_lattice_constant(structure: "ase.atoms.Atoms", engine) -> float:
    ...
    return a_0

In [ ]:
def get_lattice_constant_with_QH(
    structure: "ase.atoms.Atoms",
    temperature: list[float] | float,
    engine,
    **kwargs,
) -> list[float] | float:
    ...
    return a_0

In [ ]:
def get_lattice_constant_with_MD_NPT(
    structure: Atoms,
    temperature: list[float] | float,
    engine,
    **kwargs,
) -> list[float] | float:
    ...
    return a_0

In [ ]:
def get_lattice_constant_with_MD_NVT(
    structure: "ase.atoms.Atoms",
    temperature: list[float] | float,
    engine,
    **kwargs,
) -> list[float] | float:
    ...
    return a_0

In [ ]:
def get_deformations(structure) -> list[list[float, float, float, float, float, float]]:
    ...
    return epsilon

In [ ]:
# distribute get_stress_with_MD

def get_stress_with_MD(
    structure, temperature, strains: list[float, float, float, float, float, float], engine
):
    ...
    return sigma

def get_energy_with_MD(structure, temperature, strains, engine):
    ...
    return energy

def get_stress_with_QH(structure, temperature, strains, engine):
    ...
    return sigma

def get_energy_with_QH(structure, temperature, strains, engine):
    ...
    return energy